In [1]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes.utils import mounting_pvc # mounting_pvc 추가

DOCKER_REGISTRY = 'www.dolearn.io:30003/kade-kubeflow'

In [2]:
def train_with_fairing():
    
    import numpy as np
#     import matplotlib.pyplot as plt
    import datetime, os

    import torch
    from torchvision import transforms, datasets
    from torch.utils.data import DataLoader
    from torch.utils.tensorboard import SummaryWriter

    from torch import nn
    import torch.nn.functional as F
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using {} device".format(device))
    
    train_data = datasets.FashionMNIST(
    root="/home/jovyan/mlops-kubeflow/data/FashionMNIST",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    )

    test_data = datasets.FashionMNIST(
        root="/home/jovyan/mlops-kubeflow/data/FashionMNIST",
        train=False,
        download=True,
        transform=transforms.ToTensor(),
    )
    
    BATCH_SIZE = 32
    train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE)

    for (x_train, y_train) in train_dataloader:
        print("Shape of X [N, C, H, W]: ", x_train.shape)
        print("Shape of y: ", y_train.shape, y_train.dtype)
        break

#     plt.figure(figsize=(10, 1))
#     for i in range(10):
#         plt.subplot(1, 10, i + 1)
#         plt.imshow(x_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
#         plt.title("class: " + str(y_train[i].item()))
#         plt.axis("off")

    class NeuralNetwork(nn.Module):

        def __init__(self):
            super(NeuralNetwork, self).__init__()

            self.flatten = nn.Flatten()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 10),
                nn.ReLU()
            )

        def forward(self, x):
            x = self.flatten(x)
            logits = self.linear_relu_stack(x)
            output = F.log_softmax(logits, dim=1)
            return output

    model = NeuralNetwork().to(device)
    print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    def train(dataloader, model, loss_fn, optimizer):
        size = len(dataloader.dataset)
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            # 예측 오류 계산
            pred = model(X)
            loss = loss_fn(pred, y)

            # 역전파
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch % 500 == 0:
                loss, current = loss.item(), batch * len(X)
                # 텐서보드에 Train Loss / per epoch 로그 기록 
                writer.add_scalar('Train/Loss', loss, t+1)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            

    def test(dataloader, model, loss_fn):
        size = len(dataloader.dataset)
        num_batches = len(dataloader)
        model.eval()
        test_loss, correct = 0, 0
        with torch.no_grad():
            for X, y in dataloader:
                X, y = X.to(device), y.to(device)
                pred = model(X)
                test_loss += loss_fn(pred, y).item()
                correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
        test_accuracy = 100. * correct 
        # 텐서보드에 Test 로그 기록
        writer.add_scalar('Test/Loss', test_loss, t+1)
        writer.add_scalar('Test/Accuracy', test_accuracy, t+1)
        writer.flush()
        print(f"Test Result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    date_folder = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # 분기설정 
    if os.getenv('FAIRING_RUNTIME', None) is None:
        log_dir = "/home/jovyan/log/fit/" + date_folder
    else:
        log_dir = "/home/jovyan/job/log/fit/" + date_folder  
        
    print(f"tensorboard log dir : {log_dir}")
   
    writer = SummaryWriter(log_dir)
    epochs = 3

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)
        test(test_dataloader, model, loss_fn)


    print("Done!")

In [3]:
fairing.config.set_builder(
    'append',
    image_name='fairing-job',
    registry=DOCKER_REGISTRY,
    base_image='www.dolearn.io:30003/base/fairing-base:0.0.2')


# fairing mounting pvc 추가
notebook_volume = mounting_pvc(pvc_name="workspace-kade", 
                                pvc_mount_path="/home/jovyan") #마운트 경로  

fairing.config.set_deployer('job',
                            pod_spec_mutators=[notebook_volume], #마운트 할 볼륨명
                            cleanup=True) # 잡을 실행후 완료시 Job 삭제할지의 여부를 결정


[W 220123 12:43:56 utils:51] The function mounting_pvc has been deprecated,                     please use `volume_mounts`


In [4]:
if __name__ == '__main__':
    print('remote train_with_fairing')
    remote_train = fairing.config.fn(train_with_fairing)
    remote_train()

[I 220123 12:43:56 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f55ecfb22e8>
[I 220123 12:43:56 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f5610776940>
[I 220123 12:43:56 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f5603b1dd30>
[W 220123 12:43:56 append:52] Building image using Append builder...
[I 220123 12:43:56 base:112] Creating docker context: /tmp/fairing_context_49gw_9s7
[W 220123 12:43:56 base:99] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 220123 12:43:56 docker_creds_:234] Loading Docker credentials for repository 'www.dolearn.io:30003/base/fairing-base:0.0.2'


remote train_with_fairing
Image name :  www.dolearn.io:30003/kade-kubeflow/fairing-job:9D07133D


[W 220123 12:43:57 append:56] Image successfully built in 0.8322863881476223s.
[W 220123 12:43:57 append:98] Pushing image www.dolearn.io:30003/kade-kubeflow/fairing-job:9D07133D...
[I 220123 12:43:57 docker_creds_:234] Loading Docker credentials for repository 'www.dolearn.io:30003/kade-kubeflow/fairing-job:9D07133D'
[W 220123 12:43:57 append:85] Uploading www.dolearn.io:30003/kade-kubeflow/fairing-job:9D07133D


Image name :  www.dolearn.io:30003/kade-kubeflow/fairing-job:9D07133D


[I 220123 12:43:57 docker_session_:280] Layer sha256:4bf23ae646f0b9d8e07bf427c69c82f208bb57a8b297507d9b8b6fa23b725711 exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:191b6069f9932358899ffcc3f45c41a0f5f2731b948236a6553484caaf989794 exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:f571d568b0961b0954a50f361ad842acab3b6e4b21a27430e172a1f0d5aca5db exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:a4dd3c805ec24b016ac8a3869add24541829736c312b65bd49d3b2af7501f897 exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:3cf8fb62ba5ffb221a2edb2208741346eb4d2d99a174138e4afbb69ce1fd9966 exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:f5098a9bf4490bccac9085b1bf9c54baf3015333c40fb6685889a9785b7388ee exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:f22ccc0b8772d8e1bcb40f137b373686bc27427a70c0e41dd22b38016e09e7e0 exists, skipping
[I 220123 12:43:58 docker_session_:280] Layer sha256:20

ConnectionResetError: [Errno 104] Connection reset by peer